In [1]:
import json
import pandas as pd
import glob
from pandas.io.json import json_normalize
pd.options.display.max_rows = 200
pd.options.display.max_columns = 999

## Process the Atti Dirigenti Files

In [7]:
path = '../regione-toscana/atti_dirigenti/regione-toscana*'

In [8]:
files = glob.glob(path)
files[:5]

['regione-toscana/atti_dirigenti/regione-toscana-result-type_atti_dirigenti-year_2016-from_14000-to_14200.json',
 'regione-toscana/atti_dirigenti/regione-toscana-result-type_atti_dirigenti-year_2005-from_2200-to_2400.json',
 'regione-toscana/atti_dirigenti/regione-toscana-result-type_atti_dirigenti-year_2017-from_10600-to_10800.json',
 'regione-toscana/atti_dirigenti/regione-toscana-result-type_atti_dirigenti-year_2017-from_13600-to_13800.json',
 'regione-toscana/atti_dirigenti/regione-toscana-result-type_atti_dirigenti-year_2005-from_5000-to_5200.json']

In [9]:
columns = ['MT.CODICE_PRATICA', 'MT.DATA_ATTO', 'MT.DESCRIZIONE_TIPO_PRATICA', 'MT.ID_TIPO', 'MT.ID_TIPO_PRATICA', 
           'MT.LINK_CONTENUTO_BINARIO', 'MT.OGGETTO', 'MT.PERSONA', 'MT.STATUS', 'MT.UFFICIO_AC',
           'MT.UFFICIO_DG', 'MT.UFFICIO_SE', 'UE']

update_columns = [v[3:] for v in columns[:-1]]
update_columns.append('DETAIL_URL')

In [10]:
df_list = []

In [11]:
 def unwrap_list(x):
        if type(x) == list:
            return x[0]
        else:
            return x

In [12]:
for file in files:
    with open(file, 'r') as r:
        data = json.loads(r.read())
        flat_records = json_normalize(data['GSP']['RES']['R'])
        df_atti = flat_records[columns]
        df_atti.columns = update_columns
        df_atti = df_atti.applymap(unwrap_list)
        df_list.append(df_atti)

In [13]:
len(df_list)

771

In [14]:
df_atti = pd.concat(df_list)

In [15]:
df_atti.shape

(153206, 13)

## Definizione Uffici

- DG: uffici Direzione Generale
- SE: Uffici Settore

SE e' un sotto ufficio della DG

In [16]:
df_atti.head()

,CODICE_PRATICA,DATA_ATTO,DESCRIZIONE_TIPO_PRATICA,ID_TIPO,ID_TIPO_PRATICA,LINK_CONTENUTO_BINARIO,OGGETTO,PERSONA,STATUS,UFFICIO_AC,UFFICIO_DG,UFFICIO_SE,DETAIL_URL
0,20160140002016AD00000017201,2016-12-19,Atto dei dirigenti,ADN,MON,http://www.regione.toscana.it/bancadati/atti/C...,DGR 968/07 e s.m.i. Accreditamento degli organ...,013910,2,00000,50125,05873,http://www.regione.toscana.it/bancadati/atti/D...
1,20160140012016AD00000017466,2016-12-21,Atto dei dirigenti,ADN,MON,http://www.regione.toscana.it/bancadati/atti/C...,REG. CEE 2080/92 - AIUTI ALLE MISURE FORESTALI...,005595,2,00000,50117,05015,http://www.regione.toscana.it/bancadati/atti/D...
2,20160140022016AD00000017198,2016-12-19,Atto dei dirigenti,ADN,MON,http://www.regione.toscana.it/bancadati/atti/C...,"Servizi autorizzati ex art.14 L.R. 42/98 ""Norm...",019075,2,00000,50121,04838,http://www.regione.toscana.it/bancadati/atti/D...
3,20160140032016AD00000017208,2016-12-19,Atto dei dirigenti,ADN,MON,http://www.regione.toscana.it/bancadati/atti/C...,"L.R. 1/2006, DCR 3/2012, L.R. 1/2015 art.29, c...",020415,2,00000,50117,05018,http://www.regione.toscana.it/bancadati/atti/D...
4,20160140042016AD00000017268,2016-12-20,Atto dei dirigenti,ADN,MON,http://www.regione.toscana.it/bancadati/atti/C...,L.R. n. 87/1997 - Pubblicazione sul BURT dell'...,010826,2,00000,50116,05895,http://www.regione.toscana.it/bancadati/atti/D...


In [18]:
df_atti['STATUS'] = df_atti['STATUS'].astype('int64')

In [4]:
df_atti['DATA_ATTO'] = pd.to_datetime(df_atti.DATA_ATTO)

In [5]:
df_atti.dtypes

CODICE_PRATICA                      object
DATA_ATTO                   datetime64[ns]
DESCRIZIONE_TIPO_PRATICA            object
ID_TIPO                             object
ID_TIPO_PRATICA                     object
LINK_CONTENUTO_BINARIO              object
OGGETTO                             object
PERSONA                             object
STATUS                               int64
UFFICIO_AC                          object
UFFICIO_DG                          object
UFFICIO_SE                          object
DETAIL_URL                          object
dtype: object

In [6]:
df_atti.to_pickle('regione-toscana/atti-dirigenti.pkl')